# Spatialisation des naissances: production des données

In [1]:
### Noter qu'il faudra avoir installé toutes les librairies nécessaires
# dans l'environnement conda utilisé pour exécuter ce carnet
import pandas as pd
import geopandas as gpd
#from shapely.geometry import LineString
from geopandas.tools import sjoin
from geodatasets import get_path
#import networkx as nx
import matplotlib.pyplot as plt
import plotly.express as px
from itables import init_notebook_mode, show
import numpy as np
#import seaborn as sns
import json
import IPython

In [2]:
### Librairies déjà installées avec Python
import pprint
import sqlite3 as sql
import sys
from importlib import reload


In [3]:
### Importer un module de fonctions crées ad hoc
##  ATTENTION : le fichier 'sparql_functions.py' doit se trouver 
#   dans un dossier qui se situe dans le chemin ('path') de recherche
#   vu par le présent carnet Jupyter afin que
#   l'importation fonctionne correctement

# Add parent directory to the path
sys.path.insert(0, '..')

### If you want to add the parent-parent directory,
sys.path.insert(0, '../..')


import sparql_functions as spqf

## Production des données



In [5]:
## define SPARQL enpoint
endpoint = "https://query.wikidata.org/sparql"

In [6]:
query = """
SELECT DISTINCT ?item ?year ?birthPlace ?birthPlaceLabel  ?coordinates # ?type ?typeLabel
        WHERE {
            {
            {?item wdt:P106 wd:Q169470}
          UNION 
          {?item wdt:P106 wd:Q11063}
            UNION
              {?item wdt:P106 wd:Q155647}
              }
          
          ?item wdt:P31 wd:Q5;  # Any instance of a human.
              wdt:P569 ?birthDate;
                wdt:P19 ?birthPlace.
          OPTIONAL {
            ?birthPlace wdt:P625 ?coordinates.
              # ?birthPlace  wdt:P31 ?type
          }
        BIND(REPLACE(str(?birthDate), "(.*)([0-9]{4})(.*)", "$2") AS ?year)
        FILTER(xsd:integer(?year) > 1350 && xsd:integer(?year) < 1976 )
          
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
        } 
     ORDER BY ?year ?item
"""

In [ ]:
### Executer la requête avec les fonctions de la librairie locale
qr = spqf.get_json_sparql_result(endpoint,query)

In [12]:
r = [l for l in spqf.sparql_result_to_list(qr)]
print(len(r))
r[:3]

22307


[['http://www.wikidata.org/entity/Q855257',
  '1355',
  'http://www.wikidata.org/entity/Q2683',
  'Parma',
  'Point(10.328 44.801472222)'],
 ['http://www.wikidata.org/entity/Q4121967',
  '1359',
  'http://www.wikidata.org/entity/Q85',
  'Cairo',
  'Point(31.235833333 30.044444444)'],
 ['http://www.wikidata.org/entity/Q9168741',
  '1360',
  'http://www.wikidata.org/entity/Q266623',
  'Szatmár County',
  'Point(22.4667 47.6833)']]

In [13]:
### Créer un DataFrame à partir du résultat
df_r = pd.DataFrame(r)
df_r.columns = ['personUri', 'birthYear', 'placeUri', 'placeLabel', 'placeCoord']
df_r.head()

,personUri,birthYear,placeUri,placeLabel,placeCoord
0,http://www.wikidata.org/entity/Q855257,1355,http://www.wikidata.org/entity/Q2683,Parma,Point(10.328 44.801472222)
1,http://www.wikidata.org/entity/Q4121967,1359,http://www.wikidata.org/entity/Q85,Cairo,Point(31.235833333 30.044444444)
2,http://www.wikidata.org/entity/Q9168741,1360,http://www.wikidata.org/entity/Q266623,Szatmár County,Point(22.4667 47.6833)
3,http://www.wikidata.org/entity/Q1386256,1364,http://www.wikidata.org/entity/Q40738,Bursa,Point(29.080538888 40.191330555)
4,http://www.wikidata.org/entity/Q326797,1368,http://www.wikidata.org/entity/Q327764,Prachatice,Point(13.997533003 49.013013368)


In [14]:
df_r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22307 entries, 0 to 22306
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   personUri   22307 non-null  object
 1   birthYear   22307 non-null  object
 2   placeUri    22307 non-null  object
 3   placeLabel  22307 non-null  object
 4   placeCoord  22307 non-null  object
dtypes: object(5)
memory usage: 871.5+ KB


## Insérer les données dans la base de données

In [4]:
### Se connecter à la base de données dans laquelle on va insérer
# le résultat de la requête SPARQL
cn = sql.connect('../../data/astronomers_import.db')
cn

In [16]:
### Créer une nouvelle table contenant le DataFrame
# Si on tente de la recréer, alor qu'elle existe déjà,
# un message d'erreur est renvoyé
try:
    l = df_r.to_sql(name='wdt_person_birth_place', con=cn, if_exists='fail')
except Exception as e:
    print('Erreur: ',  e)